**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from scipy.optimize import curve_fit

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/p5q7gzupa2ndw55/sigmoid_regression_data.csv?dl=1", directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }
def fit_poly(X, Y, d):
    x_min = np.min(X)
    x_max = np.max(X)
    
    polynomial_features = PolynomialFeatures(degree=d)
    X_poly = polynomial_features.fit_transform(X)
    linreg = LinearRegression()
    linreg = linreg.fit(X_poly, Y)
    
    xx = np.linspace(x_min, x_max, 250).reshape((-1, 1))
    xx_poly = polynomial_features.transform(xx)
    yy = linreg.predict(xx_poly)
    
    plt.figure()
    plt.scatter(X, Y)
    plt.grid(ls='--')
    plt.xlabel('x')
    plt.ylabel('y')

    plt.plot(xx, yy, 'r')
    plt.title("degree {}".format(d))    
    plt.savefig("output/poly_{}_fit.pdf".format(d), bbox_inches="tight", pad_inches=0)

## Polynomická regresia

### Lineárna regresia sa nehodí na všetko

Lineárna regresia sa nedá vhodne aplikovať na ľubovoľné dáta. Napríklad:



In [ ]:
#@title -- Linear Regression on Polynomial Data -- { display-mode: "form" }
X = np.arange(-5, 10, 0.2)
Y = (X - 2 * (X ** 2) + 0.5 * (X ** 3)
        + np.random.normal(0, 15, len(X)))

X = X.reshape([-1, 1])
Y = Y.reshape([-1, 1])

linreg = LinearRegression()
linreg = linreg.fit(X, Y)
y_lin = linreg.predict(X)

plt.figure()
plt.scatter(X, Y, s=10)
plt.plot(X, y_lin, 'r')
plt.grid(ls='--')
plt.xlabel('x')
plt.ylabel('y')

plt.savefig("output/poly_linfit.pdf", bbox_inches="tight", pad_inches=0)

Ako vidno, lineárna závislosť v tomto prípade charakter dát dobre nevystihuje.

### Aplikácia polynomickej regresie

Ďalším z mnohých iných typov regresie je polynomická regresia, kde sa dáta prekladajú polynómom určitého stupňa. Regresný model vyzerá nasledovne:
\begin{equation}
\hat y = a_0 + a_1 x + a_2 x^2 + ... + a_n x^n
\end{equation}

Dobrá správa je, že polynomiálna regresia sa dá realizovať úplne rovnakým spôsobom ako lineárna regresia – stačí regresný problém reinterpretovať tak, že vstupom lineárneho regresného modelu nebude priamo $x$, ale vektor jeho mocnín.

Matica vstupov pre lineárny regresný model bude mať potom tvar
\begin{equation}
X = \left(
\begin{matrix}
1 & x_1 & x_1^2 & ... & x_1^n \
1 & x_2 & x_2^2 & ... & x_2^n \
\vdots & \vdots & \vdots & \ddots & \vdots \
1 & x_m & x_m^2 & ... & x_m^n
\end{matrix}
\right),
\end{equation}
kde $n$ je stupeň polynómu.

Balíček `sklearn` obsahuje objekt `PolynomialFeatures`, ktorý pomáha predspracovať vstupy práve do takého tvaru. Keď ho spojíme s lineárnym regresorom, získame vlastne polynomický regresor. Napríklad pre polynóm stupňa 3:



In [ ]:
model = make_pipeline(PolynomialFeatures(degree=3), LinearRegression())
model.fit(X, Y)

In [ ]:
#@title -- Regression Curve vs. Data -- { display-mode: "form" }
x_min = np.min(X)
x_max = np.max(X)
xx = np.linspace(x_min, x_max, 250).reshape((-1, 1))
yy = model.predict(xx)

plt.figure()
plt.scatter(X, Y, s=10)
plt.plot(xx, yy, 'r')
plt.grid(ls='--')
plt.xlabel('x')
plt.ylabel('y')

plt.savefig("output/polyfit.pdf", bbox_inches="tight", pad_inches=0)

Ako vidno, výsledky sú omnoho lepšie: pretože pôvodné dáta pochádzajú práve z polynómu stupňa 3.

### Ako voliť stupeň polynómu?

Pri voľbe stupňa polynómu treba dať pozor na to, čo sa vždy uvažuje aj pri strojovom učení všeobecne. Model by mal byť na jednej strane dosť zložitý na to, aby vyjadril zákonitosti prítomné v dátovej množine, ale nie taký zložitý, aby si dáta zapamätal naspamäť, pretože potom pravdepodobne nebude správne zovšeobecňovať. Ak je model príliš jednoduchý, hovoríme, že ide o **underfitting** , ak príliš zložitý, o **overfitting** .

Problém budeme ilustrovať na jednoduchom príklade:



In [ ]:
#@title -- Data Loading and Preprocessing; X, Y -- { display-mode: "form" }
df = pd.read_csv("data/sigmoid_regression_data.csv")

# we specify the inputs and the outputs
categorical_inputs = []
numeric_inputs = ['x']
output = ['y']

# we create the pipeline
input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

# we fit and apply the pipeline
X = input_preproc.fit_transform(df[categorical_inputs+numeric_inputs])
Y = df[output].values

# we plot the data for visual inspection
plt.scatter(X, Y, marker='x', label="training data")
plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()
plt.savefig("output/regression_data.pdf", bbox_inches='tight', pad_inches=0)

Skúsme teraz použiť polynómy rôznych stupňov (funkcia `fit_poly` bola zadefinovaná na začiatku v bloku s pomocnými funkciami):



In [ ]:
fit_poly(X, Y, 2)
fit_poly(X, Y, 5)
fit_poly(X, Y, 7)
fit_poly(X, Y, 11)

Ako vidno, polynómy nízkeho stupňa nevedia vystihnúť tvar závislosti, z ktorej sú pôvodné body odčítané. Ak zvolíme naopak príliš vysoký stupeň polynómu, ten bude bodmi prechádzať veľmi presne, ale medzi nimi sa nebude správať rozumne.

Pochopiteľne, chybou je vôbec aproximovať takúto krivku pomocou polynomickej regresie. Pôvodná závislosť sa nápadne podobá na sigmoidnú krivku, bolo by teda ďaleko rozumnejšie aproximovať ich pomocou nej:



In [ ]:
#@title -- Fitting the Data using the Sigmoid Curve -- { display-mode: "form" }
def sigmoid(x, x0, k, a, c):
    y = a / (1 + np.exp(-k*(x-x0))) + c
    return y

x_min = np.min(X)
x_max = np.max(X)
xx = np.linspace(x_min, x_max, 250).reshape((-1, 1))

popt, pcov = curve_fit(sigmoid, X.reshape(-1), Y.reshape(-1))
yy = sigmoid(xx, *popt)

plt.figure()
plt.scatter(X, Y)
plt.grid(ls='--')
plt.xlabel('x')
plt.ylabel('y')
plt.plot(xx, yy, 'r')
plt.savefig("output/sigmoid_fit.pdf", bbox_inches="tight", pad_inches=0)

Ako vidno, výsledky sú v tomto prípade neporovnateľne lepšie.

